Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# From 3D Segmentation to Immersive Visualization: A Complete Workflow for Mesh Conversion, USD Export, and NVIDIA Omniverse Integration
In this tutorial, we’ll cover:

- Utilizing 3D Segmentation Results: How to extract and prepare segmentation data from VISTA-3D or MAISI for mesh conversion.
- Converting to Mesh Format: Step-by-step instructions on transforming segmentation results into mesh models.
- Exporting to USD: A guide to exporting meshes as Universal Scene Description (USD) files, optimized for Omniverse workflows.
- Visualizing in NVIDIA Omniverse: Instructions on importing USD files into Omniverse for high-quality 3D visualization and manipulation.
This end-to-end process enables efficient, high-quality visualization in NVIDIA Omniverse from raw segmentation data.

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!apt update
!apt install -y libgl1-mesa-glx

## Setup imports

In [2]:
import os
import tempfile

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Generate synthetic data from MAISI


In [4]:
from monai.bundle import download

download(name="maisi_ct_generative", bundle_dir=root_dir)

2024-11-13 08:00:17,717 - INFO - --- input summary of monai.bundle.scripts.download ---
2024-11-13 08:00:17,721 - INFO - > name: 'maisi_ct_generative'
2024-11-13 08:00:17,723 - INFO - > bundle_dir: '/workspace/Data'
2024-11-13 08:00:17,725 - INFO - > source: 'monaihosting'
2024-11-13 08:00:17,727 - INFO - > remove_prefix: 'monai_'
2024-11-13 08:00:17,728 - INFO - > progress: True
2024-11-13 08:00:17,729 - INFO - ---




maisi_ct_generative_v0.4.5.zip: 13.0GB [09:43, 23.8MB/s]                                


2024-11-13 08:10:07,983 - INFO - Downloaded: /workspace/Data/maisi_ct_generative_v0.4.5.zip
2024-11-13 08:10:07,984 - INFO - Expected md5 is None, skip md5 check for file /workspace/Data/maisi_ct_generative_v0.4.5.zip.
2024-11-13 08:10:07,984 - INFO - Writing into directory: /workspace/Data.


In [ ]:
from monai.bundle.scripts import create_workflow

bundle_root = os.path.join(root_dir, "maisi_ct_generative")
workflow = create_workflow(config_file=os.path.join(bundle_root,"configs/inference.json"), workflow_type="inference", bundle_root=bundle_root)
workflow.run()

In [3]:
import nibabel as nib
import numpy as np
import trimesh
from skimage import measure

input_nii_path = "/workspace/Data/maisi_ct_generative/datasets/IntegrationTest-AbdomenCT.nii.gz"
def nii_to_obj(input_nii_path, output_obj_path):
    # Load the NIfTI file
    nii_img = nib.load(input_nii_path)
    nii_data = nii_img.get_fdata()

    # Threshold the NIfTI data to create a binary mask (assumes non-zero values are the object)
    threshold = 0.5  # Adjust threshold as necessary
    binary_mask = nii_data > threshold

    # Find the vertices and faces of the surface mesh using marching cubes
    verts, faces, _, _ = measure.marching_cubes(binary_mask, level=0)

    # Create a Trimesh object
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)

    # Export the mesh to an OBJ file
    mesh.export(output_obj_path)
    print(f"OBJ file saved to {output_obj_path}")

# Example usage
output_obj_path = '/workspace/Data/maisi_ct_generative/datasets/glTF/maisi/IntegrationTest-AbdomenCT/output_file.obj'  # Replace with the desired output path
nii_to_obj(input_nii_path, output_obj_path)

OBJ file saved to /workspace/Data/maisi_ct_generative/datasets/glTF/maisi/IntegrationTest-AbdomenCT/output_file.obj


In [2]:
import nibabel as nib
import numpy as np
import trimesh
from skimage import measure
def save_obj_file(data, output_obj_file):
    # Step 2: Generate a mesh using the marching cubes algorithm
    # iso_value = np.mean(data)  # Adjust this value based on your data
    vertices, faces, normals, values = measure.marching_cubes(
        volume=data
    )

    # Step 3: Create a Trimesh mesh
    mesh = trimesh.Trimesh(
        vertices=vertices, faces=faces, vertex_normals=normals
    )

    # Step 4: Define and assign material properties
    material = trimesh.visual.material.SimpleMaterial(
        name='SurfaceMaterial',
        ambient=[1.0, 0.0, 0.0],   # Red ambient color
        diffuse=[1.0, 0.0, 0.0],   # Red diffuse color
        specular=[1.0, 1.0, 1.0],  # White specular color
        specular_weight=0.5
    )
    mesh.visual.material = material

    mesh.export(output_obj_file)

In [5]:
import nibabel as nib
import numpy as np
from monai.transforms import LoadImage, SaveImage
import glob
import os
import nrrd
# from OpenAnatomyExport import OpenAnatomyExportLogic
path_to_maisi_preds = "/workspace/Data/maisi_ct_generative/datasets/"
path_to_obj_gltfs = "/workspace/Data/maisi_ct_generative/datasets/monai/"

all_preds = glob.glob(os.path.join(path_to_maisi_preds, "*.nii.gz"))
color_map = {
    1: [1.0, 0.0, 0.0],
    2: [0.0, 1.0, 0.0],
    3: [0.0, 0.0, 1.0],
    4: [1.0, 1.0, 0.0],
    5: [1.0, 0.0, 1.0],
    6: [0.0, 1.0, 1.0],
    7: [1.0, 0.5, 0.0],
    8: [0.5, 0.0, 1.0],
    9: [0.0, 0.5, 1.0],
    10: [0.5, 1.0, 0.0],
    11: [1.0, 0.0, 0.5],
    12: [0.0, 1.0, 0.5],
    13: [0.5, 0.5, 0.0],
    14: [0.5, 0.0, 0.5],
    15: [0.0, 0.5, 0.5],
    16: [0.75, 0.75, 0.75],
    17: [0.25, 0.25, 0.25],
}

# 17 groupings that cover 101 segments/regions out of 140
labels = {
            "Liver": 1,
            "Spleen": 3,
            "Pancreas": 4,
            "Heart": 115,
            "Body": 200,
            "Gallbladder": 10,
            "Stomach": 12,
            "Small_bowel": 19,
            "Colon": 62,
            "Kidney": {"right_kidney": 5,
                       "left_kidney": 14
                       },
            "Veins": {"aorta": 6,
                      "inferior_vena_cava": 7,
                      "portal_vein_and_splenic_vein": 17,
                      "left_iliac_artery": 58,
                      "right_iliac_artery": 59,
                      "left_iliac_vena": 60,
                      "right_iliac_vena": 61,
                      "pulmonary_vein": 119,
                      "left_subclavian_artery": 123,
                      "right_subclavian_artery": 124,
                      "superior_vena_cava": 125,
                      "brachiocephalic_trunk": 109,
                      "left_brachiocephalic_vein": 110,
                      "right_brachiocephalic_vein": 111,
                      "left_common_carotid_artery": 112,
                      "right_common_carotid_artery": 113,
                      },
            "Lungs": {"left_lung_upper_lobe": 28,
                      "left_lung_lower_lobe": 29,
                      "right_lung_upper_lobe": 30,
                      "right_lung_middle_lobe": 31,
                      "right_lung_lower_lobe": 32
                      },
            "Spine": {
                    "vertebrae_L6": 131,
                    "vertebrae_L5": 33,
                    "vertebrae_L4": 34,
                    "vertebrae_L3": 35,
                    "vertebrae_L2": 36,
                    "vertebrae_L1": 37,
                    "vertebrae_T12": 38,
                    "vertebrae_T11": 39,
                    "vertebrae_T10": 40,
                    "vertebrae_T9": 41,
                    "vertebrae_T8": 42,
                    "vertebrae_T7": 43,
                    "vertebrae_T6": 44,
                    "vertebrae_T5": 45,
                    "vertebrae_T4": 46,
                    "vertebrae_T3": 47,
                    "vertebrae_T2": 48,
                    "vertebrae_T1": 49,
                    "vertebrae_C7": 50,
                    "vertebrae_C6": 51,
                    "vertebrae_C5": 52,
                    "vertebrae_C4": 53,
                    "vertebrae_C3": 54,
                    "vertebrae_C2": 55,
                    "vertebrae_C1": 56,
                    "sacrum": 97,
                    "vertebrae_S1": 127,
                    },
            "Ribs": {
                    "left_rib_1": 63,
                    "left_rib_2": 64,
                    "left_rib_3": 65,
                    "left_rib_4": 66,
                    "left_rib_5": 67,
                    "left_rib_6": 68,
                    "left_rib_7": 69,
                    "left_rib_8": 70,
                    "left_rib_9": 71,
                    "left_rib_10": 72,
                    "left_rib_11": 73,
                    "left_rib_12": 74,
                    "right_rib_1": 75,
                    "right_rib_2": 76,
                    "right_rib_3": 77,
                    "right_rib_4": 78,
                    "right_rib_5": 79,
                    "right_rib_6": 80,
                    "right_rib_7": 81,
                    "right_rib_8": 82,
                    "right_rib_9": 83,
                    "right_rib_10": 84,
                    "right_rib_11": 85,
                    "right_rib_12": 86,
                    "costal_cartilages": 114,
                    "sternum": 122,
                    },
            "Shoulders": {
                        "left_scapula": 89,
                        "right_scapula": 90,
                        "left_clavicula": 91,
                        "right_clavicula": 92
            },
            "Hips": {
                    "left_hip": 95,
                    "right_hip": 96
                    },
            "Back_muscles": {
                            "left_gluteus_maximus": 98,
                            "right_gluteus_maximus": 99,
                            "left_gluteus_medius": 100,
                            "right_gluteus_medius": 101,
                            "left_gluteus_minimus": 102,
                            "right_gluteus_minimus": 103,
                            "left_autochthon": 104,
                            "right_autochthon": 105,
                            "left_iliopsoas": 106,
                            "right_iliopsoas": 107
                            }
}

for pred in all_preds:
    filename = os.path.basename(pred).split('.')[0]
    path_filename = os.path.join(path_to_maisi_preds, filename) # for NRRD files
    obj_gltf_path = os.path.join(path_to_obj_gltfs, filename) # for independent OBJ files
    if not os.path.exists(path_filename):
        os.makedirs(path_filename)

    if not os.path.exists(obj_gltf_path):
        os.makedirs(obj_gltf_path)
    print(f"Running for file: {filename}")

    orig_seg = LoadImage()(pred)

    final_seg = np.zeros_like(orig_seg, dtype=np.uint8)
    organ_np = np.zeros_like(orig_seg, dtype=np.uint8)

    print("Merging labels ...")

    save_trans = SaveImage(path_filename, output_ext="nrrd", output_dtype=np.uint8)
    save_trans.set_options(write_kwargs = {"compression":True})
    labels_dict = dict()
    meshes = []
    scene = trimesh.Scene()
    for j, (organ_name, labelVal) in enumerate(labels.items(), start=1):
        print(f"Assigning index {j} to label {organ_name}")
        if isinstance(labelVal, dict):
            for _, i in labelVal.items():
                final_seg[orig_seg == i] = j
                organ_np[orig_seg == i] = j
        else:
            final_seg[orig_seg == labelVal] = j
            organ_np[orig_seg == labelVal] = j
        save_trans(organ_np[None], filename=f"{path_to_maisi_preds}monai/{organ_name}")
    
        try:
            verts, faces, norms, vals = measure.marching_cubes(organ_np, level=j-1)
        except:
            print(f"Error in marching cubes for {organ_name}")
            continue    
        # create mesh
        mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=norms)
        print(mesh)
        mesh = mesh.smoothed(filter='laplacian', iterations=10, lamb=0.5)
        print('after smooth', mesh)
        color = color_map.get(j, [1.0, 1.0, 1.0])
        material = trimesh.visual.material.SimpleMaterial(
            name=organ_name,
            diffuse=color,
            ambient=color,
            specular=[1.0, 1.0, 1.0],
            specular_weight=0.5
        )
        
        mesh.visual.material = material
        
        scene.add_geometry(mesh, geom_name=organ_name)
        break

    save_trans(final_seg[None], meta_data=orig_seg.meta, filename=f"{path_to_maisi_preds}monai/all_organs")
    # save_obj_file(final_seg, os.path.join(obj_gltf_path, f"all_organs.obj"))
    scene.export(os.path.join(obj_gltf_path, f"all_organs.obj"))
    print(f"Saved whole segmentation {filename}.nrrd")

Running for file: IntegrationTest-AbdomenCT
Merging labels ...
Assigning index 1 to label Liver
2024-11-18 06:31:49,659 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/Liver.nrrd


<trimesh.Trimesh(vertices.shape=(203432, 3), faces.shape=(411228, 3))>
after smooth <trimesh.Trimesh(vertices.shape=(787030, 3), faces.shape=(411228, 3))>
2024-11-18 06:31:57,288 INFO image_writer.py:197 - writing: /workspace/Data/maisi_ct_generative/datasets/monai/all_organs.nrrd
Saved whole segmentation IntegrationTest-AbdomenCT.nrrd


In [4]:
import vtk
import os

# Function to perform segmentation-to-mesh conversion and smoothing
def convert_segmentation_to_mesh(segmentation_path, output_folder, filename, smoothing_factor=0.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Step 1: Load segmentation (binary labelmap, e.g., NRRD file)
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(segmentation_path)
    reader.Update()

    # Step 2: Create Closed Surface Representation using vtkDiscreteFlyingEdges3D
    flying_edges = vtk.vtkDiscreteFlyingEdges3D()
    flying_edges.SetInputConnection(reader.GetOutputPort())
    flying_edges.ComputeGradientsOff()
    flying_edges.ComputeNormalsOff()
    flying_edges.SetValue(0, 1)  # Assuming label 1 for segmentation surface
    flying_edges.Update()

    decimation_filter = vtk.vtkDecimatePro()
    decimation_filter.SetInputConnection(flying_edges.GetOutputPort())
    decimation_filter.SetFeatureAngle(60)
    decimation_filter.SplittingOff()
    decimation_filter.PreserveTopologyOn()
    decimation_filter.SetMaximumError(1)
    decimation_filter.SetTargetReduction(0.9)  # Adjust reduction level (0.0 to 1.0)
    decimation_filter.Update()

    # Step 3: Smooth the resulting mesh
    smoothing_filter = vtk.vtkWindowedSincPolyDataFilter()
    numberOfIterations = int(20 + smoothing_factor * 40)
    passBand = pow(10.0, -4.0 * smoothing_factor)
    smoothing_filter.SetInputConnection(decimation_filter.GetOutputPort())
    smoothing_filter.SetNumberOfIterations(numberOfIterations)  # Smooth iterations
    smoothing_filter.SetPassBand(passBand)  # Smoothing passband
    smoothing_filter.FeatureEdgeSmoothingOff()
    smoothing_filter.NonManifoldSmoothingOn()
    smoothing_filter.NormalizeCoordinatesOn()
    smoothing_filter.Update()

    # Step 4: Generate normals for better shading
    normals_filter = vtk.vtkPolyDataNormals()
    normals_filter.SetInputConnection(smoothing_filter.GetOutputPort())
    normals_filter.ConsistencyOn()
    normals_filter.SplittingOff()
    normals_filter.Update()

    polydata = normals_filter.GetOutput()
    # Step 5: Export the smoothed mesh to glTF
    writer = vtk.vtkOBJWriter()
    writer.SetFileName(os.path.join(output_folder, filename))
    writer.SetInputData(polydata)  # Use the polydata object
    writer.Write()

    print(f"Mesh successfully exported to {filename}")

# Input segmentation file (e.g., NRRD file path) and output folder
seg_path = "/workspace/Data/maisi_ct_generative/datasets/monai/Liver.nrrd"
output_folder = "/workspace/Data/maisi_ct_generative/datasets/monai/"
filename = "segmentation_model_new-03-09.obj"
# Perform conversion
convert_segmentation_to_mesh(seg_path, output_folder, filename)

Mesh successfully exported to segmentation_model_new-03-09.obj


In [5]:
from pxr import Usd, UsdGeom, Gf, Sdf
import sys

def convert_obj_to_usd(obj_filename, usd_filename):
    # Create a new USD stage
    stage = Usd.Stage.CreateNew(usd_filename)

    # Define a mesh at the root of the stage
    mesh = UsdGeom.Mesh.Define(stage, '/RootMesh')

    # Lists to hold OBJ data
    vertices = []
    normals = []
    texcoords = []
    face_vertex_indices = []
    face_vertex_counts = []

    # Mapping for OBJ indices (since they can be specified per face-vertex)
    vertex_indices = []
    normal_indices = []
    texcoord_indices = []

    # Read the OBJ file
    with open(obj_filename, 'r') as obj_file:
        for line in obj_file:
            if line.startswith('v '):
                # Vertex position
                _, x, y, z = line.strip().split()
                vertices.append((float(x), float(y), float(z)))
            elif line.startswith('vn '):
                # Vertex normal
                _, nx, ny, nz = line.strip().split()
                normals.append((float(nx), float(ny), float(nz)))
            elif line.startswith('vt '):
                # Texture coordinate
                _, u, v = line.strip().split()
                texcoords.append((float(u), float(v)))
            elif line.startswith('f '):
                # Face
                face_elements = line.strip().split()[1:]
                vertex_count = len(face_elements)
                face_vertex_counts.append(vertex_count)
                for elem in face_elements:
                    indices = elem.split('/')
                    # OBJ indices are 1-based; subtract 1 for 0-based indexing
                    vi = int(indices[0]) - 1
                    ti = int(indices[1]) - 1 if len(indices) > 1 and indices[1] else None
                    ni = int(indices[2]) - 1 if len(indices) > 2 and indices[2] else None
                    face_vertex_indices.append(vi)
                    if ni is not None:
                        normal_indices.append(ni)
                    if ti is not None:
                        texcoord_indices.append(ti)

    # Set the mesh's points
    mesh.CreatePointsAttr([Gf.Vec3f(*v) for v in vertices])

    # Set the face vertex indices and counts
    mesh.CreateFaceVertexIndicesAttr(face_vertex_indices)
    mesh.CreateFaceVertexCountsAttr(face_vertex_counts)

    # Optionally set normals if they exist
    if normals and normal_indices:
        # Reorder normals according to face vertices
        ordered_normals = [normals[i] for i in normal_indices]
        mesh.CreateNormalsAttr([Gf.Vec3f(*n) for n in ordered_normals])
        mesh.SetNormalsInterpolation('faceVarying')  # Adjust based on how normals are specified

    # Optionally set texture coordinates if they exist
    if texcoords and texcoord_indices:
        # Reorder texcoords according to face vertices
        ordered_texcoords = [texcoords[i] for i in texcoord_indices]
        stPrimvar = mesh.CreatePrimvar('st', Sdf.ValueTypeNames.TexCoord2fArray, UsdGeom.Tokens.faceVarying)
        stPrimvar.Set([Gf.Vec2f(*tc) for tc in ordered_texcoords])

    # Save the stage
    stage.GetRootLayer().Save()

obj_filename = "/workspace/Data/maisi_ct_generative/datasets/monai/segmentation_model_new-03-09.obj"
usd_filename = "/workspace/Data/maisi_ct_generative/datasets/monai/segmentation_model_new-03-09.obj.usda"

convert_obj_to_usd(obj_filename, usd_filename)

In [ ]:
from visualization import DisplayUSD

usd_filename = "/workspace/Data/maisi_ct_generative/datasets/monai/segmentation_model_new-03-09.obj.usda"
DisplayUSD(usd_filename, show_usd_code=True)